## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-07-10-30-00 +0000,wsj,Why Nvidia and Other AI Stocks Have Lost Their...,https://www.wsj.com/finance/investing/why-nvid...
1,2025-12-07-10-30-00 +0000,wsj,Five Reasons Investors Are Feeling Good About ...,https://www.wsj.com/finance/investing/stock-ma...
2,2025-12-07-10-22-52 +0000,bbc,Soldiers in Benin say they have seized power f...,https://www.bbc.com/news/articles/c62v7n9wzkyo...
3,2025-12-07-10-02-48 +0000,nyt,Superpower Competition: The Missing Chapter in...,https://www.nytimes.com/2025/12/07/us/politics...
4,2025-12-07-10-02-39 +0000,nyt,Netflix Deal for Warner Bros. Adds New Twist t...,https://www.nytimes.com/2025/12/07/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2413/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,23
166,fire,13
31,new,10
137,hong,7
211,deadly,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
93,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...,63
121,2025-12-06-18-10-02 +0000,nypost,Deadly Hong Kong fire came after residents wer...,https://nypost.com/2025/12/06/world-news/hong-...,50
161,2025-12-06-11-45-00 +0000,nypost,"Zelensky, Trump admin have ‘constructive’ phon...",https://nypost.com/2025/12/06/world-news/russi...,48
113,2025-12-06-19-04-27 +0000,wapo,Trump urges a new vaccine schedule. Here’s wha...,https://www.washingtonpost.com/world/2025/12/0...,45
41,2025-12-07-06-45-47 +0000,bbc,Hong Kong votes in election as city mourns dea...,https://www.bbc.com/news/articles/c3r7plnwr0zo...,42


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
93,63,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...
81,37,2025-12-06-23-09-00 +0000,wsj,Defense Secretary Pete Hegseth defended the U....,https://www.wsj.com/politics/national-security...
121,37,2025-12-06-18-10-02 +0000,nypost,Deadly Hong Kong fire came after residents wer...,https://nypost.com/2025/12/06/world-news/hong-...
52,34,2025-12-07-03-46-56 +0000,nypost,Warner Bros. Discovery CEO’s bidding war destr...,https://nypost.com/2025/12/06/business/warner-...
114,31,2025-12-06-18-58-09 +0000,bbc,Can you ban kids from social media? Australia ...,https://www.bbc.com/news/articles/crk7xgzj8y8o...
88,26,2025-12-06-21-41-29 +0000,nypost,Christmas returns to Bethlehem with tree-light...,https://nypost.com/2025/12/06/world-news/chris...
146,26,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
102,25,2025-12-06-19-52-09 +0000,nypost,San Diego agrees to $30M settlement for family...,https://nypost.com/2025/12/06/us-news/30-milli...
123,25,2025-12-06-18-00-00 +0000,wsj,His Grandfather’s Funeral Felt So…Cold. So He ...,https://www.wsj.com/business/entrepreneurship/...
147,23,2025-12-06-14-31-08 +0000,cbc,Russia unleashes massive drone and missile att...,https://www.cbc.ca/news/world/russia-ukraine-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
